In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.models import model_from_yaml
import keras.layers
from keras.optimizers import Adam,RMSprop
import numpy as np
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import glob
from skimage import color, io
from skimage.transform import resize
import keras

In [ ]:
# Input data from set of images
X_train_g = np.zeros((80000, 32, 32, 3), dtype = "float")
index = 0

#--------------------------------------------------------
# Pass path for positive gan image folder below
for filepath in glob.iglob('data_train/positive_L/*'):
# Pass path for positive gan image folder above
#--------------------------------------------------------

    image = resize(io.imread(filepath), (32, 32))
    X_train_g[index] = np.array(image).reshape(32, 32, 3) 
    index += 1
    if index == 79999:
        break


In [ ]:
# Sample function for training discriminator
def sample_for_d(X_train_g, sample_size, generator):
    # Generated Images
    random_input = np.random.normal(0, 1, size = (sample_size, 100))
    X1 = generator(random_input)
    Y1 = np.zeros((sample_size, 1))

    # Real Images
    index = np.random.randint(79999, size = (sample_size))
    X2 = X_train_g[index]
    Y2 = np.ones((sample_size, 1))

    return [X1, Y1, X2, Y2]

In [ ]:
# Sample function for training generator
def sample_for_g(X_train_g, sample_size):
    X = np.random.normal(0, 1, size = (sample_size, 100))
    Y = np.ones((sample_size,1))

    return [X, Y]

In [ ]:
# Sample function for training inverse generator
def sample_for_ig(X_train_g, sample_size, generator):
    X = np.zeros((sample_size, 32, 32, 3))
    Y = np.zeros((sample_size, 32, 32, 3))
    for i in range(0,sample_size):
        count = np.random.randint(79999)
        X[i] = X_train_g[count]
        Y[i] = X_train_g[count]

    return [X, Y]

In [ ]:
# Functions to save images and monitor training
def save_imgs(epoch):
    r, c = 2, 2
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/gan_1_%d.png" % epoch)
    plt.close()

def save_imgs2(epoch, image):
    r, c = 2, 2
    noise = invGenerator(image.reshape([1,32,32,3]))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            if i == 0:
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i,j].axis('off')
            else:
                axs[i,j].imshow(image, cmap='gray')
                axs[i,j].axis('off')
    fig.savefig("images/gan_2_%d.png" % epoch)
    plt.close()

In [ ]:
#Inverse Generative Model
img_shape = (32,32,3)
input_shape = (100,)
invGenerator = keras.models.Sequential([
    keras.layers.Conv2D(256, (4,4), strides=(2,2), padding='same', input_shape = img_shape),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(256, (4,4), strides=(2,2), padding='same'),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(256, (4,4), strides=(2,2), padding='same'),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(100)
])
invGenerator.summary()


In [ ]:
#Generative Model
img_shape = (32,32,3)
input_shape = (100,)
generator = keras.models.Sequential([
    keras.layers.Dense(4 * 4 * 256, input_shape = input_shape),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Reshape((4, 4, 256)),
    keras.layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(3, (3,3), activation='relu', padding='same')
])
generator.summary()


In [ ]:
#Discriminative Model
layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(256, (3,3), strides=(2,2), padding='same'),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(1, activation = "sigmoid")
])
discriminator.summary()

In [ ]:
# Compilation of first level gan (Training for first constraint)
discriminator.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5), metrics=["accuracy"])
generator.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5))

discriminator.trainable = False
gan1 = keras.models.Sequential([generator, discriminator])
gan1.compile(loss ="binary_crossentropy", optimizer=Adam(0.0002, 0.5), metrics=["accuracy"])
gan1.summary()

In [ ]:
# Training Procedure
for i in range(100000):

    discriminator.trainable = True
    [X_batch_d, Y_batch_d, X_batch_d2, Y_batch_d2] = sample_for_d(X_train_g, 32, generator)
    loss1 = discriminator.train_on_batch(X_batch_d, Y_batch_d)
    loss2 = discriminator.train_on_batch(X_batch_d2, Y_batch_d2)
    # print(f"{(loss1[1] + loss2[1])/ 2}")

    discriminator.trainable = False
    [X_batch_g, Y_batch_g] = sample_for_g(X_train_g, 64)
    loss = gan1.train_on_batch(X_batch_g, Y_batch_g)
    # print(loss[0])

    print(i)
    if i%1000 == 0:
        save_imgs(i)


In [ ]:
# Compilation of second level gan (Training for second constraint)
generator.trainable = False
gan2 = keras.models.Sequential([invGenerator, generator])
gan2.compile(loss = "mean_squared_error", optimizer = Adam(0.0003, 0.5))
gan2.summary()


In [ ]:
# Training Procedure
print("Inverse Generator")
loss = gan2.fit(X_train_g, X_train_g, epochs = 10, batch_size = 100)
save_imgs2(0, X_train_g[8])

In [ ]:
# Save gan2 as encoder
model_yaml = gan2.to_yaml()
with open("encoder.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
gan2.save_weights("encoder.h5")
print("Saved model to disk")